# Optimization

of linear-quadratic problems.

Many optimization problems in finance and econometrics involve linear-quadratic objectives/constraints. This notebook illustrates how the package [Convex.jl](https://github.com/jump-dev/Convex.jl) can be used for this. The example is (for pedagogical reasons) the same as in the other notebooks on optimization. Otherwise, the methods illustrated here are well suited for cases when the objective involves the portfolio variance ($ w'\Sigma w $) or when the estimation problem is based on minimizing the sum of squared residuals ($u'u$). The notebook also uses [SCS.jl](https://github.com/jump-dev/SCS.jl) (for the optimization algorithm).

As an extra, the last section of the notebook introduces the [Clarabel.jl](https://github.com/oxfordcontrol/Clarabel.jl) optimization package. It is well suited for linear-quadratic problems.

## Load Packages and Utility Functions

In [1]:
using Printf, LinearAlgebra, Convex, SCS

include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

## A Linear-Quadratic Minimization Problem

with/without constraints.

We specify a matrix $Q$ and a vector $q$ and write the loss function as $\theta'Q \theta + q' \theta$ where $\theta$ are the choice variables. 

We consider several cases below: no restrictions on $\theta$, bounds on $\theta$, a linear equality restriction and a non-linear inequality restriction.

In [2]:
Q = [1  0;           #we want to minimize θ'Q*θ + q'θ,
     0  16]          #this is the same as minimizing (x-2)^2 + (4y+3)^2
q = [-4, 24]

2-element Vector{Int64}:
 -4
 24

## Unconstrained Minimization

In [3]:
n  = length(q)
θ  = Variable(n)              #define the choice variables
L1 = quadform(θ,Q)            #part 1 of the objective, θ'Q*θ
L2 = dot(q,θ)                 #part 2, q'θ

problem = minimize(L1+L2)
solve!(problem,SCS.Optimizer;silent_solver = true)
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(θ) : NaN

println("Unconstrained minimization: the solution should be (2,-3/4)")
printmat(b_sol)

Unconstrained minimization: the solution should be (2,-3/4)
     2.000
    -0.750



## Constrained Minimization

In [4]:
c1 = [2.75 <= θ[1],θ[2] <= -0.3]     #bounds on the solution

problem = minimize(L1+L2,c1)
solve!(problem,SCS.Optimizer;silent_solver = true)
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(θ) : NaN

println("with bounds on the solution: the solution should be (2.75,-0.75)")
printmat(b_sol)

with bounds on the solution: the solution should be (2.75,-0.75)
     2.750
    -0.750



In [5]:
c2 = dot([1,2],θ) == 3     #equality constraint

problem = minimize(L1+L2,c2)
solve!(problem,SCS.Optimizer;silent_solver = true)
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(θ) : NaN

println("equality constraint: the solution should be (4,-1/2)")
printmat(b_sol)

equality constraint: the solution should be (4,-1/2)
     4.000
    -0.500



In [6]:
c3 = θ[2] + square(θ[1]-4) <= 0      #non-linear inequality constraint

problem = minimize(L1+L2,c3)
solve!(problem,SCS.Optimizer;silent_solver = true)
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(θ) : NaN

println("non-linear inequality constraint: the solution should be close to (3.1,-0.79)")
printmat(b_sol)

non-linear inequality constraint: the solution should be close to (3.1,-0.79)
     3.112
    -0.789



# The Clarabel.jl Optimization Package (extra)

The [Clarabel.jl](https://github.com/oxfordcontrol/Clarabel.jl) package is tailor made for solving linear-quadratic problems (and more). It solves problems of the type

$\min 0.5\theta' P \theta + q' \theta$ subject to $A \theta \leq b$ and/or $=b$.

The next cells will load the necessary packages, define a convenience function (to facilitate calling on the optimization algorithm) and then redo some of the previous examples. The convenience function rewrites the problem on the following form:

$\min 0.5\theta' P \theta + q' \theta$ subject to $l \leq A \theta \leq u$.

(This form is also used by the OSQP.jl package.)

(The package could be used with `Convex.jl` (instead of `SCS.jl`), but we here istead choose to call on the optimization routines directly.)

In [7]:
using LinearAlgebra, SparseArrays, Clarabel

In [8]:
"""
    DoClarabel(P,q,A,l,u,settings=Clarabel.Settings())


Utility function to help setting up the Clarabel minimisation problem according
to the OSQP API. Basically, it transforms to Float64 and sparse (for P and A)
and creates the cones from the information vectors `l` and `u`.

It solves `min 0.5θ'P*θ + q'θ` subject to `l <= A*θ <= u`. To get an equality constraint,
set `l[i] == u[i]`.


# Input
- `P::Matrix`:    nxn
- `q::Vector`:    n-vector
- `A::Matrix`:    Kxn
- `l::Vector`:    K-vector, lower bounds
- `u::Vector`:    K-vector, upper bounds
- `settings::Clarabel.Settings`: eg. `Clarabel.Settings(verbose = true)`

"""
function DoClarabel(P,q,A,l,u,settings=Clarabel.Settings())

  !(l<=u) && error("u <= l is not satisfied")

  P2 = sparse(Float64.(P))              #convert to Float64 (and sparse for P and A)
  q2 = Float64.(q)

  ve = l .== u                          #indices of equality restrictions
  vi = .!ve                             #indices of inequality restrictions

  A2    = sparse(Float64.(vcat(A[ve,:],A[vi,:],-A[vi,:])))
  b2    = Float64.(vcat(u[ve],u[vi],-l[vi]))
  
  cones = Clarabel.SupportedCone[]      #build cones
  for i = 1:length(b2)
    i <= sum(ve) ? push!(cones,Clarabel.ZeroConeT(1)) : push!(cones,Clarabel.NonnegativeConeT(1))
  end

  v = .!isinf.(b2)                             #cut out all  constraints involving -Inf,Inf
  (A2,b2,cones) = (A2[v,:],b2[v],cones[v])

  model = Clarabel.Solver()
  Clarabel.setup!(model, P2, q2, A2, b2, cones, settings)
  result = Clarabel.solve!(model)

  x = string(result.status) == "SOLVED" ? result.x : NaN

  #fnO = (;P=P2,q=q2,A=A2,b=b2,cones)

  return x, result

end

DoClarabel

In [9]:
# Unconstrained Minimization

n = length(q)
A = zeros(1,n)                  #effectively no restriction
l = [0]
u = [0]

settings = Clarabel.Settings(verbose = false)
(b_sol,result) = DoClarabel(2*Q,q,A,l,u,settings)   #notice 2*Q

println("Unconstrained minimization: the solution should be (2,-3/4)")
printmat(b_sol)

Unconstrained minimization: the solution should be (2,-3/4)
     2.000
    -0.750



In [10]:
# Constrained Minimization

A = [1 0;
      0 1]
l = [2.75,-Inf]
u = [Inf,-0.3]

(b_sol,result) = DoClarabel(2*Q,q,A,l,u,settings)
println("with bounds on the solution: the solution should be (2.75,-0.75)")
printmat(b_sol)

with bounds on the solution: the solution should be (2.75,-0.75)
     2.750
    -0.750



In [11]:
A = [1 2]                    #equality constraint
l = [3]
u = [3]

(b_sol,result) = DoClarabel(2*Q,q,A,l,u,settings)
println("equality constraint: the solution should be (4,-1/2)")
printmat(b_sol)

equality constraint: the solution should be (4,-1/2)
     4.000
    -0.500

